### 설치 – install



In [ ]:
!pip install selenium

In [ ]:
pip install ipywidgets

### 드라이버 – driver

In [ ]:
!pip install chromedriver_autoinstaller

### 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import os
import math 

import requests
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

# Selenium
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver import ActionChains as AC # ActionChains are a way to automate low level interactions such as mouse movements, mouse button actions, key press, and context menu interactions.
from selenium.webdriver.common.by import By       # Set of supported locator strategies.
from selenium.webdriver.common.keys import Keys   # Set of special keys codes
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait    # Constructor, takes a WebDriver instance and timeout in seconds.

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')


### 브라우저 열기 (chrome)

In [2]:
# 크롬창에서 멜론 발라드 부분 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 크롬 옵션
options = webdriver.ChromeOptions()  
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800, 1200") # window-size -> 기본 : 1920,1080

driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1")
time.sleep(3)

### 🧞‍♂️ 전체 코드 (가사 없는 곡 PASS 가능) : 인기순 추출

In [ ]:
songId = []
artist_names = []
songs = []
lyrics = []


### 페이지 이동 (1~1095 pages)
for j in tqdm(range(500,600)) :  # PAGE 범위 조정할 것 (~1095 페이지까지 있음) 
 
    print(f'----------{j+1}page 크롤링 중----------')
    #차단 막기
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # url 설정 (해당 페이지에 대한 url)
    page_url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={1+(50*j)}'
    
    # url 접속
    driver.get(page_url)
    time.sleep(5)


    ### URL 이용하여 해당 페이지의 곡 번호 추출하기 (50 songs/page)
    # 가사 아이콘 확인하고 곡번호 url 확인하기
    elements = driver.find_elements(By.CSS_SELECTOR, "a.btn.button_icons.type03.song_info")
    
    product_id = []
    for ele in elements:
        thing = ele.get_attribute('href')
        p = re.compile('[0-9]{1,8}')
        song_id = p.findall(thing)[0]
        # print(song_id)
        product_id.append(song_id)

    print(product_id)

    ### 곡 번호(songId)로 돌리기 in page
    for i in tqdm_notebook(product_id) :   # product_id
        print(f'----------{product_id.index(i)}번째 노래를 크롤링 중입니다.----------')
        #차단 막기
        seed = np.random.randint(100)
        np.random.seed(seed)
        a = np.random.randint(5)
        time.sleep(a)

        # url 설정 (해당 곡에 대한 url)
        url = f'https://www.melon.com/song/detail.htm?songId={i}'

        # url 접속
        driver.get(url)
        time.sleep(3)    

        # 가수명 크롤링
        artist_name = driver.find_element(By.CSS_SELECTOR, ".artist").text
        print(artist_name)
        artist_names.append(artist_name)
        
        # 곡명 크롤링
        song_name = driver.find_element(By.CSS_SELECTOR, "div.song_name").text
        print(song_name)
        songs.append(song_name)

            
        # 가사 있을 때
        try:
            # 펼치기 아이콘 클릭
            driver.find_element(By.CSS_SELECTOR, "button.button_more.arrow_d").click()
            time.sleep(3)

            # 가사 크롤링
            lyric = driver.find_element(By.ID, "d_video_summary").text
            print(lyric)
            lyrics.append(lyric)

        # 가사 없을 때
        except:
            lyric = "[가사 준비중]"
            print(lyric)
            lyrics.append(lyric)
            
    page_coll = {}

    page_coll['가수'] = artist_names
    page_coll['제목'] = songs
    page_coll['가사'] = lyrics
    page_coll

    target_df = pd.DataFrame.from_dict(page_coll, orient='index')
    df = target_df.transpose()
    df

    # csv 파일로 저장
    df.to_csv(f"melon_ballad({j+1}).csv", encoding='utf-8-sig')
        
            
# 가사 없는 곡 예시
# https://www.melon.com/song/detail.htm?songId=35413703 밤의 해변에서 혼자 (연주곡) / 초야
# https://www.melon.com/song/detail.htm?songId=35319615 꿈속으로 / Hope
# https://www.melon.com/song/detail.htm?songId=30811172 민들레 홀씨 되어 (cover.ver) / Various Artists

